# Understanding
## TensorFlow tf.io API for Python

### Go through UnderstandingImages FIRST.   Then this worksheet.   
UnderstandingImages will get the image data - a prerequisite for this notebook.

https://www.tensorflow.org/api_docs/python/tf/io

be careful with version v1.14 & v2.0

### TensorFlow Python Ops
a bunch of helper functions  
https://github.com/tensorflow/tensorflow/tree/master/tensorflow/python/ops

tensorflow.python.framework
https://github.com/tensorflow/tensorflow/tree/master/tensorflow/python/framework

Why - These standard routines will help you with file I/O and creating data for TensorFlow.  Use these utilities wherever possible!!

Look at UnderstandingExample after this notebook

In [ ]:
import os, sys
import json
import numpy as np
import cv2

import tensorflow as tf
from tensorflow.python.ops import string_ops, math_ops
from matplotlib import pyplot as plt

In [ ]:
# This is needed to display the images.
%matplotlib inline

## Experiment - Eager Execution
you can run this notebook:
- without Eager Execution - you'll see Tensors but no data (because of lazy execution of the DAG)
- with Eager Execution - you'll see values

Eager Execution is default with TF 2.0

So, try  running with & without executinb the next block


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.enable_eager_execution()

In [ ]:
# GLOBALs

PROJECT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(PROJECT_DIR, "data/jpeg_images")
SAMPLE_IMAGE = "20190710_variety_1562781002.jpg"

MODEL_PATH = os.path.join(PROJECT_DIR, "trained_model/export/Servo/1564865938")
LABEL_MAP = os.path.join(PROJECT_DIR, "code/cfa_prod_label_map.pbtxt")

# you can get data using the TrainModel_Step1_Local notebook
TEST_TFRECORDS_PATH =  os.path.join(PROJECT_DIR, "code/tfrecords/test/")
                                    
SAMPLE_IMAGE = "/home/ec2-user/SageMaker/ssd-dag/data/jpeg_images/20190710_variety_1562781002.jpg"

### tf.io.gfile

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/io/gfile

Note that common file i/o commands can be accomplished with this class

In [ ]:
# simple directory list
file_list = tf.io.gfile.listdir(IMAGE_DIR)
file_list_length = len(file_list)
print ("directory list length:", file_list_length)

In [ ]:
# directory list based on a glob
glob = IMAGE_DIR + '/*9.jpg'
print (glob)
file_list = tf.io.gfile.glob(glob)
file_list_length = len(file_list)
print ("directory list length:", file_list_length)
sample_image = file_list[0]

# note - full path 
print (sample_image)

In [ ]:
# read an image - encoded_jpg is type=bytes
# sample_image - from above, not SAMPLE_IMAGE - the global

with tf.io.gfile.GFile(sample_image, 'rb') as fid:
    encoded_jpg = fid.read()
print ("encoded_jpg:", type(encoded_jpg))  # bytes

### Read Image Comparison
- tf.io.gfile.GFile => encoded bytes
- tf.io.read_file   => Tensor
- tf.keras...       => PIL.Image.Image

In [ ]:
# tf.io.read_file
image = tf.io.read_file(sample_image)
print ("tf.io.read_file:", type(image))

### tf.keras
better for manipulating the data, shape, type

In [ ]:
# tf.keras
image = tf.keras.preprocessing.image.load_img(sample_image, target_size=[300, 300])
print (type(image))
plt.imshow(image)
plt.axis('off')

In [ ]:
# convert PIL.Image.Image => numpy array
# - Normalized
x = tf.keras.preprocessing.image.img_to_array(image)
print (type(x), x.shape)

In [ ]:
# preprocess - as req'd by MobileNet - to get add the instance dimension
x32 = tf.keras.applications.mobilenet.preprocess_input(x[tf.newaxis,...])
print ("x32:", type(x32), x32.shape, x32.dtype)
print ("one pixel RGB - normalized:", x32[0,1,1,])

In [ ]:
# unsigned 8 bit
x8 = x32.astype(np.uint8)
print ("x8:", type(x8), x8.shape, x8.dtype)
print ("one pixel RGB - uint8:", x8[0,1,1,])

## jpeg utilities

In [ ]:
# is_jpeg
# ??  is Eager Execution on ??
#  -- if NO, then the read operation creates the tensor - but it is empty (lazy)
#     and is_jpeg doesn't work
#  -- if YES, then you have an EagerTensor - with data
#     and this works

# takes a byte string
with tf.io.gfile.GFile(sample_image, 'rb') as fid:
    encoded_jpg = fid.read()

jpeg_prefix = b'\xff\xd8\xff'
print (encoded_jpg[0:3], '\n', jpeg_prefix)

print ('prefix equality test:', (encoded_jpg[:3] == jpeg_prefix))

# the source code shows:
contents = encoded_jpg

substr = string_ops.substr(contents, 0, 3)
return_value = math_ops.equal(substr, b'\xff\xd8\xff')  # name omitted
print ('return_value:', return_value)

funct_return = tf.io.is_jpeg(encoded_jpg)
print ('funct_return:', funct_return)

In [ ]:
print ("image file:", sample_image)
image_tensor = tf.io.read_file(sample_image)
print ("read_file yields:", type(image_tensor))
print ("    ", image_tensor)
print ("is_jpeg:", tf.io.is_jpeg(image_tensor))

In [ ]:
# decode_base64
# - decode byte string to a Tensor

# - ERROR - something is wrong here
#   - not dealing with it now
# decoded64_image = tf.io.decode_base64(encoded_jpg)
# print ("decoded:", type(decoded64_image))
# print (decoded64_image)

In [ ]:
# decode & crop
# - assuming this is a 300,300,3 input image
# Note:
# - result is a Tensor - shaped, type: uint8
# - assuming you can grayscale it in the operation also with channels = 1 even though input was 3
dec_crop_image = tf.io.decode_and_crop_jpeg(encoded_jpg, [100,100,20,20], channels=1)
print ("decoded & cropped")
print (type(dec_crop_image))
print (dec_crop_image)

In [ ]:
# decode_image
# - figure out the format

result = tf.io.decode_image(encoded_jpg,
                           channels=None,
                           dtype=tf.dtypes.uint8,
                           name=None,
                           expand_animations=True)
print (type(result))
print (result)

## TFRecords
